<a href="https://colab.research.google.com/github/zahra-ziaei/teaching-python-undergrade/blob/main/Homework_4_(solutions).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Backpropagation for bias vectors (1 point)

In class, we discussed a multilayer perceptron (neural network) whose layers were all "dense", i.e. the output $a^m \in \mathbb{R}^{N^m}$ of the $m$th layer is computed as 
\begin{align*}
z^m &= W^m a^{m - 1} + b^m \\
a^m &= \sigma^m(z^m)
\end{align*}
where $W^m \in \mathbb{R}^{N^m \times N^{m - 1}}$ is the weight matrix, $b^m \in \mathbb{R}^{N^m}$ is the bias vector, and $\sigma^m$ is the nonlinearity. We showed that 
$$\frac{\partial C}{\partial W^m} = \frac{\partial C}{\partial z^m} a^{m - 1 \top}$$
Show that
$$\frac{\partial C}{\partial b^m} = \frac{\partial C}{\partial z^m}$$
Hint: The derivation is almost the same as for $W$.

## Solution

From the chain rule,
$$\frac{\partial C}{\partial b^m_i} = \sum_{k = 1}^{N^m} \frac{\partial C}{\partial z^m_k} \frac{\partial z^m_k}{\partial b^m_i}$$
All terms of this sum except the one where $k = i$ will be zero, so we have
\begin{align*}
\frac{\partial z^m_i}{\partial b^m_i} &= \frac{\partial}{\partial b^m_i}\left(\sum_{l = 1}^{N^m} W^m_{il} a_l^{m - 1} + b^m_i\right) \\
&= 1, \\
\rightarrow \frac{\partial z^m_i}{\partial b^m_i} &= \begin{cases}
0 & k \ne i \\
1 & k = i
\end{cases}
\end{align*}

The original summation therefore simplifies to
$$\frac{\partial C}{\partial b^m_i} = \frac{\partial C}{\partial z^m_i}$$

# 2. MLP from scratch (3 points)

Using numpy only, implement backward pass or a sigmoid MLP. Specifically, you will need to implement this functionality in the `train` function in the `SigmoidMLP` class below. You should write numpy code to populate the two lists `weight_gradients` and `bias_gradient`, where each entry in each list corresponds to the gradient for a weight matrix or bias vector for each layer.

In [ ]:
import numpy as np

class Layer:
    def __init__(self, inputs, outputs):
        # Initialize weight matrix and bias vector
        # Getting the initialization right can be tricky, but for this problem
        # simply drawing from a standard normal distribution should work.
        self.weights = np.random.randn(outputs, inputs)
        self.biases = np.random.randn(outputs, 1)
    def __call__(self, X):
        # Compute \sigmoid(Wx + b)
        return 1/(1 + np.exp(-(self.weights.dot(X) + self.biases)))

class SigmoidMLP:

    def __init__(self, layer_widths):
        self.layers = []
        for inputs, outputs in zip(layer_widths[:-1], layer_widths[1:]):
            self.layers.append(Layer(inputs, outputs))
    
    def train(self, inputs, targets, learning_rate):
        # Forward pass - compute each layer's output and store it for later use
        layer_outputs = [inputs]
        for layer in self.layers:
            layer_outputs.append(layer(layer_outputs[-1]))
        
        # Implement backward pass to populate weight_gradients and bias_gradients
        # lists here
        cost_partials = [layer_outputs[-1] - targets]
        for layer, layer_output in zip(reversed(self.layers), reversed(layer_outputs[:-1])):
            cost_partials.append(layer.weights.T.dot(cost_partials[-1])*layer_output*(1 - layer_output))
        cost_partials.reverse()
        
        # Compute weight gradient step
        weight_gradients = []
        for cost_partial, layer_output in zip(cost_partials[1:], layer_outputs[:-1]):
            weight_gradients.append(cost_partial.dot(layer_output.T)/inputs.shape[1])
        # and biases
        bias_gradients = [cost_partial.mean(axis=1).reshape(-1, 1) for cost_partial in cost_partials[1:]]
        
        # Perform gradient descent by applying updates
        for weight_gradient, bias_gradient, layer in zip(weight_gradients, bias_gradients, self.layers):
            layer.weights -= weight_gradient*learning_rate
            layer.biases -= bias_gradient*learning_rate

    def __call__(self, inputs):
        a = inputs
        for layer in self.layers:
            a = layer(a)
        return a

def train_mlp(n_iterations, learning_rate):
    mlp = SigmoidMLP([2, 2, 1])
    inputs = np.array([[0, 1, 0, 1], 
                       [0, 0, 1, 1]])
    targets = np.array([[0, 1, 1, 0]])
    for _ in range(int(1e3)):
        mlp.train(inputs, targets, learning_rate)
    return mlp

In [ ]:
# You may need to change the n_iterations and learning_rate values
# but these worked for me
mlp = train_mlp(1000, 1.)
# The following calls should result in (approximately) 0, 1, 1, 0
# If the outputs are somewhat close, your training has succeeded!
print(mlp(np.array([0, 0]).reshape(-1, 1)))
print(mlp(np.array([0, 1]).reshape(-1, 1)))
print(mlp(np.array([1, 0]).reshape(-1, 1)))
print(mlp(np.array([1, 1]).reshape(-1, 1)))

[[0.01468419]]
[[0.98093589]]
[[0.98608727]]
[[0.01255408]]
